<a href="https://colab.research.google.com/github/fazrulrahman/Restaurant-Recommender/blob/master/Restaurant_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## loading required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

!pip install fuzzywuzzy
!pip install python-Levenshtein
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Preprocessing
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
!pip install num2words
from num2words import num2words
import re

# Distance between vector in multi-dimentional space
import os
import gensim
from gensim.models import LsiModel
from gensim import models
from gensim import corpora
from gensim.utils import lemmatize
import nltk
from gensim.parsing.preprocessing import remove_stopwords, stem_text
from gensim.parsing.preprocessing import strip_numeric
import pandas as pd
from gensim import similarities

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
## User input

# cuisine = input("What do you like to Eat(Cuisine Type):")
# budget = input("What's your bugdet for one person:")
# location = input("Preferred Location:")
# additional = input("Any other comments:")

cuisine = "Chinese"
# budget = 500
location = "mg road"
additional = "pork"

In [ ]:
## Downloading dataset
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1j-vywORX8WjldF0OsTo_fwRmebf8fGfm',
                                    dest_path='./data/RestoInfo - Recommendation.csv',
                                    unzip=False)

reviews = pd.read_csv("data/RestoInfo - Recommendation.csv")
reviews['key'] = reviews['name'] + reviews['listed_in(type)'] + reviews['listed_in(city)']

In [ ]:
# @title Funtions for Data Normalization and data cleaning
## Funtions for Data Normalization

# Restaurant Type
def atomizing_rest_type():
  temp_rest_type = reviews[['name','listed_in(type)','listed_in(city)','rest_type']]
  temp_rest_type['rest_type'] = temp_rest_type['rest_type'].str.split(',').tolist()
  temp_rest_type['rest_type'] = temp_rest_type['rest_type'].fillna('0')

  temp_rest_type['key'] = temp_rest_type['name'] + temp_rest_type['listed_in(type)'] + temp_rest_type['listed_in(city)']
  temp_rest_type1 = temp_rest_type[['key','rest_type']]

  rest_type = pd.DataFrame([[x] + [z] for x, y in temp_rest_type1.values for z in y],columns=temp_rest_type1.columns)

  rest_type = rest_type.merge(temp_rest_type[['key','name','listed_in(type)','listed_in(city)']],how = 'left',on = 'key')
  rest_type = rest_type[['key','name','listed_in(type)','listed_in(city)','rest_type']]
  return(rest_type)

# Liked Dishes
def atomizing_liked_dishes():
  temp_dish_liked = reviews[['name','listed_in(type)','listed_in(city)','dish_liked']]
  temp_dish_liked['dish_liked'] = temp_dish_liked['dish_liked'].str.split(',').tolist()
  temp_dish_liked['dish_liked'] = temp_dish_liked['dish_liked'].fillna('0')

  temp_dish_liked['key'] = temp_dish_liked['name'] + temp_dish_liked['listed_in(type)'] + temp_dish_liked['listed_in(city)']
  temp_dish_liked1 = temp_dish_liked[['key','dish_liked']]

  dish_liked = pd.DataFrame([[x] + [z] for x, y in temp_dish_liked1.values for z in y],columns=temp_dish_liked1.columns)

  dish_liked = dish_liked.merge(temp_dish_liked[['key','name','listed_in(type)','listed_in(city)']],how = 'left',on = 'key')
  dish_liked = dish_liked[['key','name','listed_in(type)','listed_in(city)','dish_liked']]
  dish_liked['dish_liked'] = pd.DataFrame([a.strip() for a in dish_liked['dish_liked']])
  return(dish_liked)


# cuisines
def atomizing_cuisines():
  temp_cuisines = reviews[['name','listed_in(type)','listed_in(city)','cuisines']]
  temp_cuisines['cuisines'] = temp_cuisines['cuisines'].str.split(',').tolist()
  temp_cuisines['cuisines'] = temp_cuisines['cuisines'].fillna('0')

  temp_cuisines['key'] = temp_cuisines['name'] + temp_cuisines['listed_in(type)'] + temp_cuisines['listed_in(city)']
  temp_cuisines1 = temp_cuisines[['key','cuisines']]

  cuisines = pd.DataFrame([[x] + [z] for x, y in temp_cuisines1.values for z in y],columns=temp_cuisines1.columns)

  cuisines = cuisines.merge(temp_cuisines[['key','name','listed_in(type)','listed_in(city)']],how = 'left',on = 'key')
  cuisines = cuisines[['key','name','listed_in(type)','listed_in(city)','cuisines']]
  return(cuisines)

# reviews_list
def atomizing_reviews_list():
  # temp_reviews_list = reviews[['name','listed_in(type)','listed_in(city)','reviews_list']]
  # temp_reviews_list['reviews_list'] = temp_reviews_list['reviews_list'].str.split(',').tolist()
  # temp_reviews_list['reviews_list'] = temp_reviews_list['reviews_list'].fillna('0')

  # temp_reviews_list['key'] = temp_reviews_list['name'] + temp_reviews_list['listed_in(type)'] + temp_reviews_list['listed_in(city)']
  # temp_reviews_list1 = temp_reviews_list[['key','reviews_list']]

  # reviews_list = pd.DataFrame([[x] + [z] for x, y in temp_reviews_list1.values for z in y],columns=temp_reviews_list1.columns)

  # reviews_list = reviews_list.merge(temp_reviews_list[['key','name','listed_in(type)','listed_in(city)']],how = 'left',on = 'key')
  # reviews_list = reviews_list[['key','name','listed_in(type)','listed_in(city)','reviews_list']]
  # return(reviews_list)
  for j in range(0,len(reviews.reviews_list[0])):
    temp_re_1 = str(reviews.reviews_list[j]).replace("\\","").replace("\'","").replace("RATEDn","")
    temp_re_2 = pd.DataFrame(re.split("\), \(",temp_re_1))

    cust_rate_1s = []
    cust_review_1s = []
    try_count = 0
    except_count = 0
    for i in range(0,len(temp_re_2[0])):
      try:
        cust_rate_1s.append(temp_re_2[0][i].split(",",1)[0])
        cust_review_1s.append(temp_re_2[0][i].split(",",1)[1])

        cust_review_dict = {'Cust_Rate': cust_rate_1s,
                            'Review': cust_review_1s
                            }

        cust_review_df_1s = pd.DataFrame(cust_review_dict,columns = {"Cust_Rate","Review"})
        cust_review_df_1s['key'] = reviews.key[j]
        try_count += 1
      except:
        except_count += 1

        pass

    if j == 0:
      cust_review_final_df = cust_review_df_1s.copy()
    else:
      cust_review_final_df = pd.concat([cust_review_final_df,cust_review_df_1s],axis = 0)


  # Remiving wrong entries
  cust_review_final_df['Cust_Rate'] = cust_review_final_df['Cust_Rate'].replace("\[\(","",regex = True)
  cust_review_final_df = cust_review_final_df[cust_review_final_df.Cust_Rate != "veg biryani n Raita)"]
  cust_review_final_df = cust_review_final_df[cust_review_final_df.Cust_Rate != "None"]
  cust_review_final_df['Cust_Rate'] = cust_review_final_df['Cust_Rate'].replace("Rated ","",regex = True).astype('float')
  cust_review_final_df['Review'] = cust_review_final_df['Review'].replace("\)\]","",regex = True)

  return(cust_review_final_df)

# menu_item
def atomizing_menu_item():
  temp_menu_item = reviews[['name','listed_in(type)','listed_in(city)','menu_item']]
  temp_menu_item['menu_item'] = temp_menu_item['menu_item'].apply(lambda x: x.replace('[','').replace(']',''))
  temp_menu_item['menu_item'] = temp_menu_item['menu_item'].apply(lambda x: x.replace("'",""))
  temp_menu_item['menu_item'] = pd.DataFrame([a.strip() for a in temp_menu_item['menu_item']])
  temp_menu_item['menu_item'] = temp_menu_item['menu_item'].str.split(',').tolist()
  temp_menu_item['menu_item'] = temp_menu_item['menu_item'].fillna('0')

  temp_menu_item['key'] = temp_menu_item['name'] + temp_menu_item['listed_in(type)'] + temp_menu_item['listed_in(city)']
  temp_menu_item1 = temp_menu_item[['key','menu_item']]

  menu_item = pd.DataFrame([[x] + [z] for x, y in temp_menu_item1.values for z in y],columns=temp_menu_item1.columns)

  menu_item = menu_item.merge(temp_menu_item[['key','name','listed_in(type)','listed_in(city)']],how = 'left',on = 'key')
  menu_item = menu_item[['key','name','listed_in(type)','listed_in(city)','menu_item']]
  menu_item['menu_item'] = pd.DataFrame([a.strip() for a in menu_item['menu_item']])
  return(menu_item)


def convert_lower_case(data):
    return np.char.lower(data)

def remov_duplicates(input): 
  
    # split input string separated by space 
    input = input.split(" ") 
  
    # joins two adjacent elements in iterable way 
    for i in range(0, len(input)): 
        input[i] = "".join(input[i]) 
  
    # now create dictionary using counter method 
    # which will have strings as key and their  
    # frequencies as value 
    UniqW = Counter(input) 
  
    # joins two adjacent elements in iterable way 
    s = " ".join(UniqW.keys()) 
    return(s)

def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    data = remov_duplicates(data)
    return data

def text_similarity(a,b):
  x = (fuzz.ratio(a,b))
  y = (fuzz.partial_ratio(a,b))
  z = (fuzz.token_sort_ratio(a,b))
  return(max(x,y,z))

def recomend_restaurant(preference):
  vec_bow = dictionary.doc2bow(preference.lower().split())

  vec_lsi = lsi[vec_bow]  # convert the query to LSI space
  index = similarities.MatrixSimilarity(lsi[doc_term_matrix])
  unsorted_similarity = index[vec_lsi]
  sorted_similarity = sorted(enumerate(unsorted_similarity), key=lambda item: -item[1])
  recommend_rest_list = []
  i = 1
  for index, similarity in sorted_similarity:
      # print(similarity, final_review_df.key[index] ,cor[index])
      recommend_rest_list.append([final_review_df.key[index],i])
      i+=1
  return(recommend_rest_list)

In [ ]:
## Data Normalization
# Restaurant type
rest_type = atomizing_rest_type()

# Liked Dish
dish_liked = atomizing_liked_dishes()

# Cusines
cuisines = atomizing_cuisines()

# Reviews list
# reviews_list = atomizing_reviews_list()
cust_review_final_df = atomizing_reviews_list()

# Menu item
menu_item = atomizing_menu_item()

# Base data
reviews = reviews[['name','online_order','book_table','rate','votes','location','approx_cost(for two people)',
                  'listed_in(type)','listed_in(city)']]

In [ ]:
dish_all_rest = pickle.load(open("dish_all_rest1.pkl",'rb'))
cuisine_all_rest = pickle.load(open("cuisine_all_rest1.pkl",'rb'))
menu_all_rest = pickle.load(open("menu_all_rest1.pkl",'rb'))

In [ ]:
# menu_item.menu_item.unique()[1:15]
# cuisines.cuisines
# dish_liked.dish_liked
dish_all_rest = []

for j in dish_liked.key.unique():
  temp_dish_1_rest = ""
  for i in dish_liked[dish_liked.key == j]["dish_liked"]:
    # temp_dish_1_rest.append(i)
    temp_dish_1_rest = temp_dish_1_rest + i + " "
  dish_all_rest.append([j,temp_dish_1_rest])
  
dish_all_rest = pd.DataFrame(dish_all_rest,columns = ['key','dish_liked'])


# Cuisine
cuisine_all_rest = []

for j in cuisines.key.unique():
  temp_cuisine_1_rest = ""
  for i in cuisines[cuisines.key == j]["cuisines"]:
    # temp_cuisine_1_rest.append(i)
    temp_cuisine_1_rest = temp_cuisine_1_rest + i + " "

  cuisine_all_rest.append([j,temp_cuisine_1_rest])

cuisine_all_rest = pd.DataFrame(cuisine_all_rest,columns = ['key','cuisines'])


# menu item
menu_all_rest = []

for j in menu_item.key.unique():
  temp_menu_1_rest = ""
  for i in menu_item[menu_item.key == j]["menu_item"]:
    # temp_menu_1_rest.append(i)
    temp_menu_1_rest = temp_menu_1_rest + i + " "

  menu_all_rest.append([j,temp_menu_1_rest])

menu_all_rest = pd.DataFrame(menu_all_rest,columns = ['key','menu_item'])

**Recommendation using NLP**

In [ ]:
# 1. Based on cuisine 
score = []
for i in cuisines.cuisines.unique():
  score.append([text_similarity(i,cuisine),i])

Matching_cuisine = pd.DataFrame(score,columns = ["score","cuisines"]).sort_values(by = "score", ascending = False)
Matching_cuisine = Matching_cuisine[Matching_cuisine.score > 50]

# Req cuisine restaurant
# cuisines
req_cuisine_rest = cuisines.merge(Matching_cuisine, how = 'inner', on = 'cuisines')
req_cuisine_rest = req_cuisine_rest[['key','cuisines']].drop_duplicates()

In [ ]:
# req_cuisine_rest

In [ ]:
# 2. Based on location
score = []
for i in cuisines['listed_in(city)'].unique():
  score.append([text_similarity(i,location),i])

Matching_location = pd.DataFrame(score,columns = ['score','location']).sort_values(by = 'score', ascending = False).head(1)[['location']]

rest_location = cuisines[['key','listed_in(city)']].drop_duplicates()

req_cuisine_loc_rest = req_cuisine_rest.merge(rest_location, how = 'left', on = 'key')
req_cuisine_loc_rest = req_cuisine_loc_rest[req_cuisine_loc_rest['listed_in(city)'].isin(Matching_location['location'])]

In [ ]:
# reviews
pickle.dump(reviews,open("./reviews.pkl", 'wb'))
!cp reviews.pkl drive/My\ Drive/Test

# pickle.dump(lsi, open("./lsi.pkl", 'wb'))


In [ ]:
# req_cuisine_loc_rest

In [ ]:
# 3. Based on user preference
# We consider review with rating 4 and above for customer preference matching
Req_reviews = cust_review_final_df[cust_review_final_df.Cust_Rate >= 4.0]
unique_stores = Req_reviews.key.unique()

In [ ]:
# Taking all the key words in the review for all the restaurant
final_reviews = {'rest_key':'rest_key_words'}

for j in unique_stores:
  temp_review_1 = ""
  for i in set(Req_reviews[Req_reviews.key == j].Review.to_list()):
    temp_review_1 += i
  temp_review_1 = preprocess(temp_review_1)

  final_reviews[str(j)] = temp_review_1

final_review_df = [(k, v) for k, v in final_reviews.items()] 
final_review_df = pd.DataFrame(final_review_df,columns = ['key','rev'])

In [ ]:
# final_review_df

temp_All_key_words = final_review_df.merge(dish_all_rest, how = 'outer', on = 'key')
temp_All_key_words1 = temp_All_key_words.merge(cuisine_all_rest, how = 'outer', on = 'key')
temp_All_key_words2 = temp_All_key_words1.merge(menu_all_rest, how = 'outer', on = 'key')

In [ ]:
temp_All_key_words2[temp_All_key_words2.isna()] = ""
temp_All_key_words2['new key words'] = temp_All_key_words2['rev'] + temp_All_key_words2['dish_liked'] + temp_All_key_words2['cuisines'] + temp_All_key_words2['menu_item']

In [ ]:
final_review_df = temp_All_key_words2[['key','new key words']]
final_review_df.rename(columns = {'new key words':'rev'}, inplace = True)

In [ ]:
pickle.dump(final_review_df, open("./final_review_df.pkl", 'wb'))

In [ ]:
!cp final_review_df.pkl drive/My\ Drive/Test

In [ ]:
# Modelling review in multi dimentianal space
def preprocessing(): # this should not be in code i
    for document in final_review_df.rev:
        doc = strip_numeric(stem_text(document))
        yield gensim.utils.tokenize(doc, lower=True)

texts = preprocessing() # ii
dictionary = corpora.Dictionary(texts) # iii # dictionaly to be pickled

doc_term_matrix = [dictionary.doc2bow(tokens) for tokens in preprocessing()] 
tfidf = models.TfidfModel(doc_term_matrix)
corpus_tfidf = tfidf[doc_term_matrix] # iv # to be pickeled

lsi = models.LsiModel(corpus_tfidf, id2word=dictionary)  # initialize an LSI transformation

In [ ]:
# Ranking restaurant based on customer preference
# Finding distance and between documents and ranking
# additional = "pork"
Matching_rest_rank = recomend_restaurant(additional)
req_pref_rest = pd.DataFrame(Matching_rest_rank,columns= ["key","rank"]) 

In [ ]:
## Finding Top 3 restaurant to recommend
top_3_rest = req_cuisine_loc_rest.merge(req_pref_rest, how = 'left', on = 'key').sort_values(by = 'rank', ascending = True).head(3)

In [ ]:
reviews['key'] = reviews['name'] + reviews['listed_in(type)'] + reviews['listed_in(city)']
top_3_rest = top_3_rest[['key']].merge(reviews, how = 'left', on = 'key')

In [ ]:
Rank = 1
for i in top_3_rest.key:
  # print(Rank,": ",i)
  print(Rank,": ",top_3_rest[top_3_rest.key == i].name.to_list()[0])
  print("\t Cusines:",cuisines[cuisines.key == i].cuisines.to_list())
  print("\t Location:",top_3_rest[top_3_rest.key == i]['listed_in(city)'].to_list())

  Rank += 1
# cuisines[cuisines.key == i].cuisines

1 :  Hotel Shangrila
	 Cusines: ['Chinese', ' Momos']
	 Location: ['MG Road']
2 :  Harima
	 Cusines: ['Japanese']
	 Location: ['MG Road']
3 :  Hae Kum Gang
	 Cusines: ['Korean', ' Chinese', ' Japanese']
	 Location: ['MG Road']
